In [63]:
import requests
import json
import csv
import pandas as pd
import re

In [64]:
def updated_datasets():
    # This function is designed to search for updates since a given date and return the links of relevant .csv datasets
    
    year= input("Enter Year (YYYY): ")
    month= input("Enter Month (MM): ")
    day= input("Enter Date (DD): ")
    date=str(year)+'-'+str(month)+'-'+str(day)
    
    # Make request
    url='https://ws-data.nisra.gov.uk/public/api.restful/PxStat.Data.Cube_API.ReadCollection/{}/en'.format(date)
    response = requests.get(url)

    # Write response text to a JSON file
    filename = 'updated_tables_response.txt'
    with open(filename, 'w') as f:
        f.write(response.text)
        f.close()

    # Open file and read contents
    filename = 'updated_tables_response.txt'
    with open(filename, 'r') as f:
        contents = f.read()
        
    # Regex to find any https entries
    matches = re.findall('https?://[^\s]+/en', contents)
    
    # Filter out duplicate entries
    matches_no_duplicates = [*set(matches)]
    
    # Retrieve links to .json files
    json_urls = [str(match) for match in matches_no_duplicates if '/JSON-stat/1.0' in match]

    for i in range(len(json_urls)):
        print(json_urls[i])
    # Retrieve matrix codes
    new_list = [s.split('/')[-4] for s in json_urls]

    # Get label and update date from .json
    labels=[]
    updated=[]

    for i in range(0, len(json_urls)):
        df_identity = pd.read_json(json_urls[i])
        labels.append(df_identity['dataset'].label)
        updated.append(df_identity['dataset'].updated)
        df_identity = df_identity.iloc[0:0]
    
    # Make a dataframe with all the info
    data_sets_list = pd.DataFrame({'Latest Update' : updated,
                                   'Identifier' : new_list,
                                'Description' : labels,
                                'JSON' : json_urls}, 
                                columns=['Latest Update', 'Identifier','Description', 'JSON'])
    f.close()
        
    return data_sets_list

In [65]:
data_sets_list=updated_datasets()

Enter Year (YYYY): 2020
Enter Month (MM): 01
Enter Date (DD): 01
https://ws-data.nisra.gov.uk/public/api.restful/PxStat.Data.Cube_API.ReadDataset/EDATTENDLGD/JSON-stat/1.0/en
https://ws-data.nisra.gov.uk/public/api.restful/PxStat.Data.Cube_API.ReadDataset/MYE01V1/JSON-stat/1.0/en
https://ws-data.nisra.gov.uk/public/api.restful/PxStat.Data.Cube_API.ReadDataset/SDRSMOKEDEA/JSON-stat/1.0/en
https://ws-data.nisra.gov.uk/public/api.restful/PxStat.Data.Cube_API.ReadDataset/BUSINESSLGDBIG/JSON-stat/1.0/en
https://ws-data.nisra.gov.uk/public/api.restful/PxStat.Data.Cube_API.ReadDataset/MYE2021V9/JSON-stat/1.0/en
https://ws-data.nisra.gov.uk/public/api.restful/PxStat.Data.Cube_API.ReadDataset/LEHSCT/JSON-stat/1.0/en
https://ws-data.nisra.gov.uk/public/api.restful/PxStat.Data.Cube_API.ReadDataset/MYE01V3/JSON-stat/1.0/en
https://ws-data.nisra.gov.uk/public/api.restful/PxStat.Data.Cube_API.ReadDataset/INFANTMRHSCT/JSON-stat/1.0/en
https://ws-data.nisra.gov.uk/public/api.restful/PxStat.Data.Cube_A

In [66]:
data_sets_list

,Latest Update,Identifier,Description,JSON
0,2023-01-06T09:30:00Z,EDATTENDLGD,Standardised Attendance Rate – Emergency Atten...,https://ws-data.nisra.gov.uk/public/api.restfu...
1,2022-02-16T13:01:08Z,MYE01V1,Population totals - Northern Ireland,https://ws-data.nisra.gov.uk/public/api.restfu...
2,2022-11-14T13:13:41Z,SDRSMOKEDEA,Standardised Death Rate – Smoking Related Causes,https://ws-data.nisra.gov.uk/public/api.restfu...
3,2022-12-08T16:42:11Z,BUSINESSLGDBIG,Number of Businesses,https://ws-data.nisra.gov.uk/public/api.restfu...
4,2023-01-18T15:00:43Z,MYE2021V9,Population densities - Local Government Districts,https://ws-data.nisra.gov.uk/public/api.restfu...
...,...,...,...,...
104,2022-11-14T15:22:47Z,PYLLPC,Potential Years of Life Lost,https://ws-data.nisra.gov.uk/public/api.restfu...
105,2022-08-26T09:30:00Z,PORT01,Port Tonnage,https://ws-data.nisra.gov.uk/public/api.restfu...
106,2023-01-18T15:01:22Z,MYE2021V11,Median age - Northern Ireland,https://ws-data.nisra.gov.uk/public/api.restfu...
107,2022-12-09T09:30:00Z,BRESLGDHEAD,Employee Jobs,https://ws-data.nisra.gov.uk/public/api.restfu...


In [67]:
def select_data_set():
    user_choice=int(input("Enter index of selection: "))
    identifier=data_sets_list['Identifier'].iloc[user_choice]
    create_url='https://ws-data.nisra.gov.uk/public/api.restful/PxStat.Data.Cube_API.ReadDataset/{0}/CSV/1.0/en'.format(identifier)
    return create_url

In [68]:
def csv_get(input_url):
    with requests.Session() as s:
        request = s.get(input_url)
        returned = request.content
        decoded = returned.decode('utf-8')
        create = csv.reader(decoded.splitlines(), delimiter=',')
        my_list = list(create)
    df=pd.DataFrame(my_list)
    return df

In [69]:
csv_get(select_data_set())

Enter index of selection: 107


,0,1,2,3,4,5,6,7,8,9,10,11
0,"﻿""STATISTIC CODE""",Statistic,TLIST(A1),Year,LGD2014,Local Government District 2014,HEADLINE,INDUSTRY HEADLINE,GENWP,Gender and Working Pattern,UNIT,VALUE
1,EJOBS,Employee Jobs,2015,2015,N09000001,Antrim and Newtownabbey,Construction,Construction,MFT,Male Full-Time,number,2137
2,EJOBS,Employee Jobs,2015,2015,N09000001,Antrim and Newtownabbey,Construction,Construction,MPT,Male Part-Time,number,76
3,EJOBS,Employee Jobs,2015,2015,N09000001,Antrim and Newtownabbey,Construction,Construction,FFT,Female Full-Time,number,257
4,EJOBS,Employee Jobs,2015,2015,N09000001,Antrim and Newtownabbey,Construction,Construction,FPT,Female Part-Time,number,117
...,...,...,...,...,...,...,...,...,...,...,...,...
2096,EJOBS,Employee Jobs,2021,2021,N92000002,Northern Ireland,ALL,All,MFT,Male Full-Time,number,311842
2097,EJOBS,Employee Jobs,2021,2021,N92000002,Northern Ireland,ALL,All,MPT,Male Part-Time,number,67904
2098,EJOBS,Employee Jobs,2021,2021,N92000002,Northern Ireland,ALL,All,FFT,Female Full-Time,number,214154
2099,EJOBS,Employee Jobs,2021,2021,N92000002,Northern Ireland,ALL,All,FPT,Female Part-Time,number,186898
